In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from common import *

In [ ]:
from dataloaders import readmission_dataset

In [ ]:
data = readmission_dataset()

Basic Experiments
=================

Attention Comparison
====================

In [ ]:
lstm_attention_config = experiments[3](data, structured=True)
lstm_attn_eval = Evaluator(BasicCT, get_latest_model(os.path.join('outputs', lstm_attention_config['exp_config']['exp_name'])))
dev_data = data.get_data('dev', structured=True)
dev_data = data.filter_data_length(dev_data, truncate=90)
lstm_outputs = lstm_attn_eval.evaluate(dev_data, save_results=False)
del lstm_attn_eval

In [ ]:
lstm_structured_attention_config = structured_experiments[1](data, structured=True, encodings=data.structured_columns)
lstm_struct_attn_eval = Evaluator(BasicCT, 
                                  get_latest_model(os.path.join('outputs', lstm_structured_attention_config['exp_config']['exp_name'])))
dev_data = data.get_data('dev', structured=True, encodings=data.structured_columns)
dev_data = data.filter_data_length(dev_data, truncate=90)
lstm_struct_outputs = lstm_struct_attn_eval.evaluate(dev_data, save_results=False)
del lstm_struct_attn_eval

In [ ]:
from attention_comparison import get_comparative_measures

In [ ]:
measures = []
for x, a1, a2 in zip(dev_data.X, lstm_outputs['attentions'], lstm_struct_outputs['attentions']) :
    measures.append(get_comparative_measures(x, a1, a2))
measures = pd.DataFrame(measures)
y = np.array(dev_data.y)[:, 0]
measures['y'] = y

words_A = [[data.vocab.idx2word[x] for x in d] for d in list(measures['A'])]
measures['A'] = words_A

words_B = [[data.vocab.idx2word[x] for x in d] for d in list(measures['B'])]
measures['B'] = words_B

measures_0 = measures['y'] == 0
measures_1 = measures['y'] == 1

In [ ]:
lstm_max_attentions = [max(x) for x in lstm_outputs['attentions']]
lstm_struct_max_attentions = [max(x) for x in lstm_struct_outputs['attentions']]
_ = plt.hist([lstm_max_attentions, lstm_struct_max_attentions], bins=30)

In [ ]:
plt.scatter(measures['len'], measures['haus'], s=5)

In [ ]:
_ = plt.hist(measures[measures_0]['emd'], bins=30)
_ = plt.hist(measures[measures_1]['emd'], bins=30)

In [ ]:
from attention_comparison import get_comparative_measures

In [ ]:
for c in [measures_0, measures_1] :
    plt.hist([measures[c]['A&B'], measures[c]['A-B'], measures[c]['B-A']], label=['A&B', 'A-B', 'B-A'])
    plt.legend()
    plt.show()

In [ ]:
for c in [measures_0, measures_1] :
    plt.hist([measures[c]['jacc']], label=['jacc'], bins=30)
    plt.show()
    plt.scatter(measures[c]['jacc'], measures[c]['A|B'], s=1)
    plt.show()

Baselines
==========

In [ ]:
train_data = data.get_data('train', structured=True)
dev_data = data.get_data('dev', structured=True)

In [ ]:
from PatientVec.models.baselines.LR import LR, LDA

In [ ]:
lr = LR({'vocab' : data.vocab, 'stop_words' : True, 'exp_name' : data.name})
lr.train(train_data)
lr.evaluate(dev_data, save_results=True)
# lr.get_features(n=20)

In [ ]:
lda = LDA({'vocab' : data.vocab, 'stop_words' : True, 'exp_name' : data.name})
lda.train(train_data)
lda.evaluate(dev_data, save_results=True)
print(lda.get_topics(n=10))
topics = lda.get_topics(n=10)
print([topics[i] for i in np.argsort(lda.lda_classifier.coef_[0])])

In [ ]:
[topics[i] for i in np.argsort(lda.lda_classifier.coef_[0])]

In [ ]:
preds = outputs['predictions'][:, 0]

In [ ]:
from common import collapse_and_print_word_attn, print_sent_attn

In [ ]:
n = 50
collapse_and_print_word_attn(data.vocab, dev_data.X[n], outputs['word_attentions'][n])
print_sent_attn(data.vocab, dev_data.X[n], outputs['sentence_attentions'][n])

In [ ]:
outputs['sentence_attentions'][0]

In [ ]:
from scipy.stats import spearmanr, kendalltau

In [ ]:
corrs = [kendalltau(range(len(outputs['sentence_attentions'][i])), outputs['sentence_attentions'][i]) 
         for i in range(len(outputs['sentence_attentions']))]

In [ ]:
rho, leng = zip(*[(x[0], y) for x, y in zip(corrs, [len(z) for z in outputs['sentence_attentions']]) if x[0] == x[0]])

In [ ]:
plt.hist(rho, bins=30)

In [ ]:
pval, leng1 = zip(*[(x[1], y) for x, y in zip(corrs, [len(z) for z in outputs['sentence_attentions']]) if x[1] == x[1]])

Saving Models
==============

In [ ]:
from common import generate_latex_tables
keys_to_use = ['1/precision', '1/recall', '1/f1-score', 'accuracy', 'roc_auc', 'pr_auc']
generate_latex_tables(data, keys_to_use)